## Keras tutorial : Functional API 

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals


import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, Flatten, Conv2D,MaxPooling2D,concatenate,Input
from tensorflow.keras import Model
from tensorflow.keras.models import Sequential


inputs = keras.Input(shape=(784,), name='img')
x = Dense(64, activation='relu')(inputs)
x = Dense(64, activation='relu')(x)
outputs = Dense(10, activation='softmax')(x)

model = keras.Model(inputs=inputs, outputs=outputs, name='mnist_model')
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
img (InputLayer)             (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 64)                50240     
_________________________________________________________________
dense_1 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_2 (Dense)              (None, 10)                650       
Total params: 55,050
Trainable params: 55,050
Non-trainable params: 0
_________________________________________________________________


## Try 2 CNN share 1 FC in naive way 



In [0]:





inputs=keras.Input(shape=(64,64,6))
conv1=Conv2D(16, kernel_size=5, activation ='relu')(inputs)
pool1=MaxPooling2D()(conv1)
conv2=Conv2D(64, kernel_size=5, activation ='relu')(pool1)
pool2=MaxPooling2D()(conv2)
conv3=Conv2D(128, kernel_size=5, activation ='relu')(pool2)
flat1=Flatten()(conv3)

inputs2=keras.Input(shape=(64,64,6))
conv1=Conv2D(16, kernel_size=5, activation ='relu')(inputs2)
pool1=MaxPooling2D()(conv1)
conv2=Conv2D(64, kernel_size=5, activation ='relu')(pool1)
pool2=MaxPooling2D()(conv2)
conv3=Conv2D(128, kernel_size=5, activation ='relu')(pool2)
flat2=Flatten()(conv3)

merge = concatenate([flat1, flat2])
dense1 = Dense(640, activation='relu')(merge)
dense2 = Dense(2, activation='softmax')(dense1)
model = keras.Model(inputs=[inputs,inputs2], outputs=dense2)
model.summary()


keras.utils.plot_model(model, 'my_first_model.png')

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_9 (InputLayer)            (None, 64, 64, 6)    0                                            
__________________________________________________________________________________________________
input_10 (InputLayer)           (None, 64, 64, 6)    0                                            
__________________________________________________________________________________________________
conv2d_24 (Conv2D)              (None, 60, 60, 16)   2416        input_9[0][0]                    
__________________________________________________________________________________________________
conv2d_27 (Conv2D)              (None, 60, 60, 16)   2416        input_10[0][0]                   
__________________________________________________________________________________________________
max_poolin

## Kinship model ( 10 cnn share 1 FC) with func define
- Visualize  model structure 

In [0]:
inputs=[]
flats=[]

def base_cnn(imgs):
  conv1=Conv2D(16, kernel_size=5, activation ='relu')(imgs)
  pool1=MaxPooling2D()(conv1)
  conv2=Conv2D(64, kernel_size=5, activation ='relu')(pool1)
  pool2=MaxPooling2D()(conv2)
  conv3=Conv2D(128, kernel_size=5, activation ='relu')(pool2)
  flat=Flatten()(conv3)
  return flat 



for i in range(10):
  inputs.append(keras.Input(shape=(64,64,6)))
  flats.append(base_cnn(inputs[i]))
  

merge = concatenate(flats)
dense1 = Dense(640, activation='relu')(merge)
dense2 = Dense(2, activation='softmax')(dense1)
model = keras.Model(inputs=inputs, outputs=dense2)
model.summary()


keras.utils.plot_model(model, 'my_first_model.png')

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_230 (InputLayer)          (None, 64, 64, 6)    0                                            
__________________________________________________________________________________________________
input_231 (InputLayer)          (None, 64, 64, 6)    0                                            
__________________________________________________________________________________________________
input_232 (InputLayer)          (None, 64, 64, 6)    0                                            
__________________________________________________________________________________________________
input_233 (InputLayer)          (None, 64, 64, 6)    0                                            
__________________________________________________________________________________________________
input_234 

## Try Training with any data 
- mnist (but with incorrect labeling)

In [0]:

## input shape and softmax result number change
inputs=[]
flats=[]
def base_cnn(imgs):
  conv1=Conv2D(16, kernel_size=5, padding='same',activation ='relu')(imgs)
  pool1=MaxPooling2D()(conv1)
  conv2=Conv2D(64, kernel_size=5,padding='same', activation ='relu')(pool1)
  pool2=MaxPooling2D()(conv2)
  conv3=Conv2D(128, kernel_size=5,padding='same', activation ='relu')(pool2)
  flat=Flatten()(conv3)
  return flat 



for i in range(10):
  inputs.append(keras.Input(shape=(28,28,1)))
  flats.append(base_cnn(inputs[i]))
  

merge = concatenate(flats)
dense1 = Dense(640, activation='relu')(merge)
dense2 = Dense(10, activation='softmax')(dense1)
model = keras.Model(inputs=inputs, outputs=dense2)

##

(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train = x_train.reshape(60000, 28,28,1).astype('float32') / 255
x_test = x_test.reshape(10000, 28,28,1).astype('float32') / 255
x=[]
t=[]

for i in range(10):
  x.append(x_train[i*500:(i+1)*500])
  t.append(x_test[i*500:(i+1)*500])
  
y=y_train[:500]
y_t=y_test[:500]
  

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(x, y,
                    batch_size=64,
                    epochs=5)
test_scores = model.evaluate(t, y_t, verbose=0)
print('Test loss:', test_scores[0])
print('Test accuracy:', test_scores[1])

Epoch 1/5
500/500 [==============================] - 23s 45ms/sample - loss: 3.0361 - acc: 0.2340
Epoch 2/5
500/500 [==============================] - 20s 41ms/sample - loss: 1.9471 - acc: 0.4780
Epoch 3/5
500/500 [==============================] - 21s 41ms/sample - loss: 0.8881 - acc: 0.7500
Epoch 4/5
500/500 [==============================] - 20s 40ms/sample - loss: 0.5538 - acc: 0.8360
Epoch 5/5
500/500 [==============================] - 20s 40ms/sample - loss: 0.4362 - acc: 0.8740
Test loss: 0.4607326412200928
Test accuracy: 0.852


## 데이터 마련 후 사용할 부분

<추가>
The CNN is trained by back-propagation with logistic loss over
the predicted scores using the soft-max function. 
To initialize weights, we use a Gaussian distribution with zero mean and a standard deviation of 0.01. The biases are initialized as
zeros. In each iteration, we update all the weights after learning the mini-batch with the size of 128. In all layers, the momentum is set as 0.9 and the weight decay is set as 0.005. To expand the training set, we also randomly flip images and add grayscales during training.

In [0]:



x_train=[] #10 lists of imgs each list size (datanum,64,64,6)
y_train=[] #1 label per 10 imgs 0(not family) or 1(family)
x_test=[]
y_test=[]


model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(x_train, y_train,
                    batch_size=128,
                    epochs=20)
test_scores = model.evaluate(x_test,y_test , verbose=0)
print('Test loss:', test_scores[0])
print('Test accuracy:', test_scores[1])

ValueError: ignored

# 데이터 마련 

- Kinface ||  각 250쌍: Father-Son (F-S), Father-Daughter (F-D), Mother-Son (M-S), and Mother-Daughter (M-D) =>총 1000쌍


1. 각 사진 => 10개 부분 이미지으로

> 5가지:  왼쪽 눈, 오른쪽 눈, the corners of the mouth , nose
- CNN으로 얼굴의 각 포인트 점(output) 찾기
https://github.com/mysong105/Facial-Keypoint-Detection/blob/master/Facial-Keypoint-Detection.ipynb  참고 (Data: kaggle)(https://www.kaggle.com/karanjakhar/facial-keypoint-detection)
- 점들간 비율에 따라 사진 자르고 reshape (64,64,3)


>5가지(more complementary information): 전체 이미지, top-left corner, the top-right corner, bottom-left corner, bottom-right corner

2. 10개의 각 사진에 대해 가족 관계 두 사진을 합치면 label 1추가, 가족관계 아닌 경우 label 0 추가 


이후 Colab facedetect.ipynb > 
C:\Users\myson\models-master\research\object_detection\0502.ipynb


C:\Users\myson\에 다운로드 1. opencv-master (haarcascade classifier for face, eye detection ) 2. model-master (tensorflow, object_detection folder 내  파이썬 파일이 from *utils* import * 가능 )